In [ ]:
import pandas as pd
from math import log

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

print(f'>>> pd : {pd.__version__}  |  sklearn : {sklearn.__version__}')

>>> pd : 1.1.5  |  sklearn : 0.22.2.post1


## TF-IDF(Term Frequency-Inverse Document Frequency)
- TF-IDF : 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있음
- **TF-IDF = tf(d,t) * idf(d,t)** <br>
  (1) tf(d, t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수 <br>
  (2) df(t) : 특정 단어 t가 등장한 문서의 수 <br>
  (3) idf(d, t) : df(t)에 반비례하는 수 $$idf(d,t) = log({n\over1+df(t)})$$ <br>

- TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것 <br>
  (ex. the나 a와 같은 불용어는 모든 문서에 자주 등장하므로 TF-IDF 값이 낮아짐)
